# Llama Document Summary Index With Amazon Bedrock Titan Model
> *If you see errors, you may need to be allow-listed for the Bedrock models used by this notebook*

> *This notebook should work well with the **`Data Science 3.0`** kernel in SageMaker Studio*


This demo showcases the document summary index, over IRS Forms.

The document summary index will extract a summary from each document and store that summary, as well as all nodes corresponding to the document.

Retrieval can be performed through the LLM or embeddings. We first select the relevant documents to the query based on their summaries. All retrieved nodes corresponding to the selected documents are retrieved.

This notebook demonstrates invoking Bedrock models directly using the AWS SDK, but for later notebooks in the workshop you'll also need to install [LangChain](https://github.com/hwchase17/langchain):

In [2]:
%pip install langchain==0.0.309 --force-reinstall --quiet
%pip install pypdf==3.15.2 --force-reinstall --quiet
%pip install llama-index==0.8.37 --force-reinstall --quiet
%pip install sentence_transformers==2.2.2 --force-reinstall --quiet

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
daal4py 2021.6.0 requires daal==2021.4.0, which is not installed.
spyder 5.3.3 requires pyqt5<5.16, which is not installed.
spyder 5.3.3 requires pyqtwebengine<5.16, which is not installed.
apache-beam 2.50.0 requires numpy<1.25.0,>=1.14.3, but you have numpy 1.26.2 which is incompatible.
botocore 1.32.6 requires urllib3<2.1,>=1.25.4; python_version >= "3.10", but you have urllib3 2.1.0 which is incompatible.
datasets 2.14.5 requires fsspec[http]<2023.9.0,>=2023.1.0, but you have fsspec 2023.10.0 which is incompatible.
distributed 2022.7.0 requires tornado<6.2,>=6.0.3, but you have tornado 6.3.3 which is incompatible.
hdijupyterutils 0.21.0 requires pandas<2.0.0,>=0.17.1, but you have pandas 2.1.3 which is incompatible.
jupyterlab 3.4.4 requires jupyter-server~=1.16, but you have jupyter-server 2.7.3 which is inco

In [3]:
%pip install pydantic==1.10.13 --force-reinstall --quiet

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 5.3.3 requires pyqt5<5.16, which is not installed.
spyder 5.3.3 requires pyqtwebengine<5.16, which is not installed.
apache-beam 2.50.0 requires numpy<1.25.0,>=1.14.3, but you have numpy 1.26.2 which is incompatible.
llama-index 0.8.37 requires urllib3<2, but you have urllib3 2.1.0 which is incompatible.
panel 0.13.1 requires bokeh<2.5.0,>=2.4.0, but you have bokeh 3.3.0 which is incompatible.
spyder 5.3.3 requires ipython<8.0.0,>=7.31.1, but you have ipython 8.16.1 which is incompatible.
spyder 5.3.3 requires pylint<3.0,>=2.5.0, but you have pylint 3.0.1 which is incompatible.

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
%pip install sqlalchemy==2.0.21 --force-reinstall --quiet

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 5.3.3 requires pyqt5<5.16, which is not installed.
spyder 5.3.3 requires pyqtwebengine<5.16, which is not installed.
apache-beam 2.50.0 requires numpy<1.25.0,>=1.14.3, but you have numpy 1.26.2 which is incompatible.
llama-index 0.8.37 requires urllib3<2, but you have urllib3 2.1.0 which is incompatible.
panel 0.13.1 requires bokeh<2.5.0,>=2.4.0, but you have bokeh 3.3.0 which is incompatible.
spyder 5.3.3 requires ipython<8.0.0,>=7.31.1, but you have ipython 8.16.1 which is incompatible.
spyder 5.3.3 requires pylint<3.0,>=2.5.0, but you have pylint 3.0.1 which is incompatible.

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [5]:
# needed for llamaindex
%pip install openai==0.28 --force-reinstall --quiet 

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 5.3.3 requires pyqt5<5.16, which is not installed.
spyder 5.3.3 requires pyqtwebengine<5.16, which is not installed.
apache-beam 2.50.0 requires numpy<1.25.0,>=1.14.3, but you have numpy 1.26.2 which is incompatible.
botocore 1.32.6 requires urllib3<2.1,>=1.25.4; python_version >= "3.10", but you have urllib3 2.1.0 which is incompatible.
datasets 2.14.5 requires fsspec[http]<2023.9.0,>=2023.1.0, but you have fsspec 2023.10.0 which is incompatible.
distributed 2022.7.0 requires tornado<6.2,>=6.0.3, but you have tornado 6.3.3 which is incompatible.
jupyterlab 3.4.4 requires jupyter-server~=1.16, but you have jupyter-server 2.7.3 which is incompatible.
jupyterlab-server 2.10.3 requires jupyter-server~=1.4, but you have jupyter-server 2.7.3 which is incompatible.
llama-index 0.8.37 requires urllib3<2, but you h

In [6]:
import nest_asyncio

nest_asyncio.apply()

In [7]:
from llama_index import (
    SimpleDirectoryReader,
    LLMPredictor,
    ServiceContext,
    get_response_synthesizer,
    set_global_service_context
)
from llama_index.indices.document_summary import DocumentSummaryIndex

### Load Datasets

In [8]:
!mkdir -p ./data

from urllib.request import urlretrieve
urls = [
    'https://s2.q4cdn.com/299287126/files/doc_financials/2023/ar/2022-Shareholder-Letter.pdf',
    'https://s2.q4cdn.com/299287126/files/doc_financials/2022/ar/2021-Shareholder-Letter.pdf',
    'https://s2.q4cdn.com/299287126/files/doc_financials/2021/ar/Amazon-2020-Shareholder-Letter-and-1997-Shareholder-Letter.pdf',
    'https://s2.q4cdn.com/299287126/files/doc_financials/2020/ar/2019-Shareholder-Letter.pdf'
]

filenames = [
    'AMZN-2022-Shareholder-Letter.pdf',
    'AMZN-2021-Shareholder-Letter.pdf',
    'AMZN-2020-Shareholder-Letter.pdf',
    'AMZN-2019-Shareholder-Letter.pdf'
]

metadata = [
    dict(year=2022, source=filenames[0]),
    dict(year=2021, source=filenames[1]),
    dict(year=2020, source=filenames[2]),
    dict(year=2019, source=filenames[3])]

data_root = "./data/"

for idx, url in enumerate(urls):
    file_path = data_root + filenames[idx]
    urlretrieve(url, file_path)

As part of Amazon's culture, the CEO always includes a copy of the 1997 Letter to Shareholders with every new release. This will cause repetition, take longer to generate embeddings, and may skew your results. In the next section you will take the downloaded data, trim the 1997 letter (last 3 pages) and overwrite them as processed files.

In [9]:
import glob
from pypdf import PdfReader, PdfWriter

local_pdfs = glob.glob(data_root + '*.pdf')

for local_pdf in local_pdfs:
    pdf_reader = PdfReader(local_pdf)
    pdf_writer = PdfWriter()
    for pagenum in range(len(pdf_reader.pages)-3):
        page = pdf_reader.pages[pagenum]
        pdf_writer.add_page(page)

    with open(local_pdf, 'wb') as new_file:
        new_file.seek(0)
        pdf_writer.write(new_file)
        new_file.truncate()


Now that you have clean PDFs to work with, you will enrich your documents with metadata, then use a process called "chunking" to break up a larger document into small pieces. These small pieces will allow you to generate embeddings without surpassing the input limit of the embedding model.

In this example you will break the document into 1000 character chunks, with a 100 character overlap. This will allow your embeddings to maintain some of its context.

In [10]:
docs = []
for filename in filenames:
    doc = SimpleDirectoryReader(input_files=[f"data/{filename}"]).load_data()
    doc[0].doc_id = filename.replace(".pdf", "")
    docs.extend(doc)

### Build Document Summary Index

We show two ways of building the index:
- default mode of building the document summary index
- customizing the summary query


In [11]:
#### Un comment the following lines to run from your local environment outside of the AWS account with Bedrock access

#import os
#os.environ['BEDROCK_ASSUME_ROLE'] = '<YOUR_VALUES>'
#os.environ['AWS_PROFILE'] = 'bedrock-user'

In [12]:
import boto3
import json 

bedrock = boto3.client(service_name="bedrock")
bedrock_runtime = boto3.client(service_name="bedrock-runtime")

In [13]:
from llama_index import LangchainEmbedding
from langchain.llms.bedrock import Bedrock # required until llama_index offers direct Bedrock integration
from langchain.embeddings.bedrock import BedrockEmbeddings

model_kwargs_titan = { 
    "maxTokenCount": 512,
    "stopSequences": [],
    "temperature":0.0,  
    "topP":0.5
}

llm = Bedrock(model_id="amazon.titan-text-express-v1",
              client=bedrock_runtime,
              model_kwargs=model_kwargs_titan)

embed_model = LangchainEmbedding(
  BedrockEmbeddings(model_id="amazon.titan-embed-text-v1")
)

service_context = ServiceContext.from_defaults(llm=llm, 
                                               embed_model=embed_model, 
                                               chunk_size=512)
set_global_service_context(service_context)

### This will take a few minutes. Please be patient.

In [14]:
response_synthesizer = get_response_synthesizer(
    response_mode="tree_summarize", use_async=True
)
doc_summary_index = DocumentSummaryIndex.from_documents(
    docs,
    service_context=service_context,
    response_synthesizer=response_synthesizer,
)

current doc id: AMZN-2022-Shareholder-Letter
current doc id: e5456591-81c9-494d-846e-e52f3a6ed20f
current doc id: 030151ab-4ba7-4faf-a2da-85e24e8d3a19
current doc id: de0b64bd-7c54-4284-8a32-11148758e7ff
current doc id: 92689b35-fe14-4ce4-9550-eb5a2f6d150a
current doc id: a9fc7e45-7af7-45b4-abdc-c2586c6141fa
current doc id: b7b82769-bc99-4d7e-bbf5-41fbaaf68a17
current doc id: AMZN-2021-Shareholder-Letter
current doc id: 12ef9c75-af27-4b00-922f-417d07ab35d3
current doc id: d42b78e9-8697-437a-9256-cd0c5de6b69c
current doc id: 238114b8-c3ae-4a5d-88be-afb0f7270aac
current doc id: b170af2e-b3c3-4426-8296-27cd6349f4b0
current doc id: 2229a0c0-4a39-4979-91f3-58c0646de6aa
current doc id: AMZN-2020-Shareholder-Letter
current doc id: 5ec930d5-432e-49ba-8e53-046172b3ba4e
current doc id: 72777806-b7e4-4a04-97d0-87a8cb27ba37
current doc id: b8fee43a-1709-4c67-9990-c4ac161fcc05
current doc id: 0cd7c876-dee2-49ac-887a-3e824b293bf8
current doc id: 7b818758-8a79-4d84-91e8-d50ebc7e3754
current doc id: 0

In [15]:
print(len(docs))

25


In [16]:
doc_summary_index.get_document_summary("AMZN-2022-Shareholder-Letter")

"\nThe provided text is a letter from Jeff Bezos, the CEO of Amazon, to shareholders. In the letter, Bezos discusses the company's performance in 2022, despite challenging macroeconomic conditions. He highlights the company's ability to grow demand and innovate in its largest businesses to improve customer experience. Bezos also mentions the importance of making adjustments in investment decisions and the way the company will invent moving forward while preserving long-term investments. The text provides insights into Amazon's history, growth, and approach to innovation. It can answer questions about Amazon's performance in recent years, its strategies for growth, and the company's vision for the future."

In [17]:
doc_summary_index.storage_context.persist("llama_titan_index")

In [18]:
from llama_index.indices.loading import load_index_from_storage
from llama_index import StorageContext

# rebuild storage context
storage_context = StorageContext.from_defaults(persist_dir="llama_titan_index")
doc_summary_index = load_index_from_storage(storage_context)

### Perform Retrieval from Document Summary Index

We show how to execute queries at a high-level. We also show how to perform retrieval at a lower-level so that you can view the parameters that are in place. We show both LLM-based retrieval and embedding-based retrieval using the document summaries.

#### High-level Querying

Note: this uses the default, LLM-based form of retrieval

In [19]:
from utils.llama_custom_parse_choice_select_answer_fn import custom_parse_choice_select_answer_fn

query_engine = doc_summary_index.as_query_engine(
    response_mode="tree_summarize", use_async=True, similarity_top_k=10,  verbose=False,
    parse_choice_select_answer_fn=custom_parse_choice_select_answer_fn
)

In [20]:
response = query_engine.query("How has AWS evolved?")

print(response)

AWS has evolved from a simple cloud computing provider to a comprehensive cloud platform offering a wide range of services and solutions. Initially focused on providing on-demand computing resources, AWS has expanded its offerings to include serverless computing, artificial intelligence (AI), machine learning (ML), databases, analytics, networking, mobile, developer tools, IoT, security, and more.


#### LLM-based Retrieval

In [21]:
from llama_index.indices.document_summary import DocumentSummaryIndexRetriever

In [22]:
from utils.llama_custom_parse_choice_select_answer_fn import custom_parse_choice_select_answer_fn
retriever = DocumentSummaryIndexRetriever(
    doc_summary_index,
    # choice_select_prompt=choice_select_prompt,
    # choice_batch_size=choice_batch_size,
    # format_node_batch_fn=format_node_batch_fn,
    parse_choice_select_answer_fn=custom_parse_choice_select_answer_fn,
    # service_context=service_context
)

In [23]:
retrieved_nodes = retriever.retrieve("According to the documents, How has AWS evolved?")
print(len(retrieved_nodes))

0


In [24]:
for i, node in enumerate(retrieved_nodes):
    print(node.score)
    print(node.node.get_text())
    print("-----------------------------------------------------------------------------------------------------------------------------------")

In [25]:
%%time
# use retriever as part of a query engine
from llama_index.query_engine import RetrieverQueryEngine

# configure response synthesizer
response_synthesizer = get_response_synthesizer(response_mode="refine", use_async=True)

# assemble query engine
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
)

# query
response = query_engine.query("How has AWS evolved?")
print(response)

Over the past 25 years at Amazon, I’ve had the opportunity to write many narratives, emails, letters, and
keynotes for employees, customers, and partners. But, this is the first time I’ve had the honor of writing ourannual shareholder letter as CEO of Amazon. Jeff set the bar high on these letters, and I will try to keep them worth reading.

When the pandemic started in early 2020, few people thought it would be as expansive or long-running as
it’s been. Whatever role Amazon played in the world up to that point became further magnified as most physical venues shut down for long periods of time and people spent their days at home. This meant that hundreds of millions of people relied on Amazon for PPE, food, clothing, and various other items that helped them navigate this unprecedented time. Businesses and governments also had to shift, practically overnight, from working with colleagues and technology on-premises to working remotely. AWS played a major role in enabling this business co

#### Embedding-based Retrieval

In [26]:
from llama_index.indices.document_summary import DocumentSummaryIndexEmbeddingRetriever

In [27]:
retriever = DocumentSummaryIndexEmbeddingRetriever(
    doc_summary_index,
    # choice_select_prompt=choice_select_prompt,
    # choice_batch_size=choice_batch_size,
    # format_node_batch_fn=format_node_batch_fn,
    # parse_choice_select_answer_fn=parse_choice_select_answer_fn,
    #service_context=service_context
)

In [28]:
retrieved_nodes = retriever.retrieve("How has AWS evolved?")

In [29]:
len(retrieved_nodes)

3

In [30]:
for i, node in enumerate(retrieved_nodes):
    
    print(node.node.get_text())
    print("-----------------------------------------------------------------------------------------------------------------------------------")
    

work came from optimizing the connections between this large amount of infrastructure. We also continue
to improve our advanced machine learning algorithms to better predict what customers in various parts of thecountry will need so that we have the right inventory in the right regions at the right time. We’ve recentlycompleted this regional roll out and like the early results. Shorter travel distances mean lower cost to serve,less impact on the environment, and customers getting their orders faster. On the latter, we’re excited aboutseeing more next day and same-day deliveries, and we’re on track to have our fastest Prime delivery speedsever in 2023. Overall, we remain confident about our plans to lower costs, reduce delivery times, and build a
meaningfully larger retail business with healthy operating margins.
AWS has an $85B annualized revenue run rate, is still early in its adoption curve, but at a juncture where it’s
critical to stay focused on what matters most to customers over 